# 가비지 컬렉션

참고자료:
- https://winterj.me/python-gc/
- https://rushter.com/blog/python-garbage-collector/
- https://pymotw.com/3/gc/

## CPython의 GC 구성

- reference counting + cyclic(generational) garbage collector
- reference counting: 해당 객체의 참조 횟수 (0이 되면 메모리에서 할당 해제)
  - sys.getrefcount(*arg)
- cyclic garbage: 참조 횟수가 0을 도달할 수 없는 경우

In [1]:
import gc, sys, ctypes

### Reference Counting

In [2]:
foo = []
print(sys.getrefcount(foo)) # 2회: foo 변수, getrefcount() 인자

2


In [3]:
def bar(a):
    print(sys.getrefcount(a))

bar(foo) # 4회: foo 변수, getrefcount, 함수 인자, 함수 호출 스택

4


객체|값|
---|---
bar|
getrefcount|
a|foo

객체|\*args|
---|---
bar|foo

객체|\*args|
---|---
getrefcount|foo

In [4]:
print(sys.getrefcount(foo)) # 2회. 함수 scope를 벗어났기 때문에 함수 관련 인자 X

2


### Cyclic Reference

In [5]:
l = []
print(sys.getrefcount(l)) # 2회: 변수 정의, print()

l.append(l)
print(sys.getrefcount(l)) # 변수 호출, append의 값, print()

del l

2
3


In [6]:
class Foo:
    def __init__(self):
        self.x = 0

a = Foo()
b = Foo()
print('인스턴스 생성 직후, a = %d and b = %d' % (sys.getrefcount(a), sys.getrefcount(b)))

a.x = b
b.x = a
print('순환 참조, a = %d and b = %d' % (sys.getrefcount(a), sys.getrefcount(b)))

del a
print('a 삭제 후, b = %d' % sys.getrefcount(b)) # a의 경우 1
print('b는 여전히 a.x 를 참조: %s' % gc.get_referents(b))

del b

인스턴스 생성 직후, a = 2 and b = 2
순환 참조, a = 3 and b = 3
a 삭제 후, b = 3
b는 여전히 a.x 를 참조: [{'x': <__main__.Foo object at 0x00000203B0C0C2B0>}, <class '__main__.Foo'>]


In [7]:
# 주소 탐색을 위해 ctypes 모듈 사용
class PyObject(ctypes.Structure):
    _fields_ = [("refcnt", ctypes.c_long)]

gc.disable()  # Disable generational gc

lst1 = []
lst1.append(lst1)

# lst의 주소 저장
lst1_address = id(lst1)

# lst 해제
del lst1

object_1 = {}
object_2 = {}
object_1['obj2'] = object_2
object_2['obj1'] = object_1

obj_address = id(object_1)

# 참조 해제
del object_1, object_2

gc.collect()

# reference count. 메모리에 잔류하는 것을 확인
print(PyObject.from_address(obj_address).refcnt)
print(PyObject.from_address(lst1_address).refcnt)

0
0


### 파이썬 GC 모듈
- Reference Counting 모듈을 보완
  - 자동 GC를 해제하려면 gc.set_threshold(0) > gc.disable()
    - Third-party에서 enable()하는 경우 종종 발생
    - 코드 내 순환 참조 유무에 대해 체크 필수
  - 굳이 disable()할 이유가 없다
- 내부적으로 generation 및 threshold로 GC 주기 및 객체를 관리
  - 0세대: young (관리가 더 자주 일어남)
  - 각 객체는 한 세대에만 속함
  - 각 세대별 관리 주기는 gc.get_threshold() 함수를 통해 확인 가능
    - 0 -> 2세대 순
    - \# allocation - \# deallocation >= threshold\[i] -> GC
    - \# allocation - \# deallocation >= threshold\[i] * threshold\[i+1] -> 차세대 GC
    - 예) 0세대는 700번마다, 1세대는 7천번 이후, 2세대는 7만번 이후
- 각 세대별 할당 횟수 확인 후 위 기준 초과 시 gc.collect(generations=2) 수행
  - 2 -> 0세대 순
  - reference count가 0이 될 수 없는 객체 탐색
    - O: 상위 세대로 통합 / X: 메모리에서 해제

In [8]:
gc.get_threshold()

(700, 10, 10)

### 순환 참조 탐지
- 순환 참조는 Container 객체(tuple, list 등)에 의해서만 발생

#### 방법
1. 모든 컨테이너 객체 추적 (double-linked list로 관리)
2. 해당 객체의 gc_refs 필드를 레퍼런스 카운트와 동일하게 설정
3. 각 객체에서 참조하는 다른 컨테이너 객체를 찾아 해당 컨테이너의 gc_refs를 감소
  - gc_refs=0: 해당 객체들은 컨테이너 집합 내부에서 자기들끼리 참조
5. gc_refs=0인 객체를 unreachable로 마킹 후 메모리에서 해제

#### 알고리즘에 대한 상세 내용은 아래 링크 확인
  - https://github.com/python/cpython/blob/7d6ddb96b34b94c1cbdf95baa94492c48426404e/Modules/gcmodule.c#L902 (파이썬 GC모듈의 C코드)
  - http://arctrix.com/nas/python/gc/ (finalizer에 대한 이슈는 3.4에서 해결)
  - https://winterj.me/python-gc/#2-가비지-컬렉션의-작동-방식
  - 기반 언어에 따른 차이 有

In [9]:
gc.set_debug(gc.DEBUG_SAVEALL)

print(gc.get_count())

lst2 = []
lst2.append(lst2)
lst2_address = id(lst2)

del lst2

collected = gc.collect()

for item in gc.garbage:
    print(item, lst2_address == id(item))

print('# in GC: %d' % collected)

(498, 0, 0)
[[...]] True
# in GC: 1
